In [ ]:
base_path = "/content/drive/My Drive/Github/mtc-device-activation"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# Load saved data
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append(f'{base_path}/frameworks')

from casbl import casbl

data = np.load(f'{base_path}/general_model/data/general_model.npz', allow_pickle=True)
all_DeviceLocations = data['all_DeviceLocations']
num_samples = data['num_samples']
device_count = data['device_count']
sparsity = data['sparsity']

data = np.load(f'{base_path}/communication_model/data/communication_simulation_theta.npz', allow_pickle=True)
all_theta = data["all_theta"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_gamma.npz', allow_pickle=True)
all_gamma = data["all_gamma"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_z.npz', allow_pickle=True)
all_z = data["all_z"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_noise.npz', allow_pickle=True)
all_noise = data["all_noise"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_received_signal.npz', allow_pickle=True)
all_received_signal = data["all_received_signal"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_general.npz', allow_pickle=True)
snr_db_array = data["snr_db_array"]
pilot_length_array = data["pilot_length_array"]
M = data["M"]

snr_db_index = 2
snr_db=snr_db_array[snr_db_index]

max_iter = 500

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sample_range = range(600, 800)
num_samples = len(sample_range)

Sparse Bayesian Learning (SBL)

In [ ]:
# Final gamma and z results
gamma_result_all = np.empty((num_samples, len(pilot_length_array)), dtype=object)
z_result_all = np.empty((num_samples, len(pilot_length_array)), dtype=object)

In [6]:
for sample_index in tqdm(sample_range, desc="Processing Samples", position=0):
    gamma = all_gamma[sample_index]  # Ground truth gamma values (true support)
    loc = all_DeviceLocations[sample_index]
    z = all_z[sample_index]

    for pilot_length_index in tqdm(range(len(pilot_length_array)), desc="Processing Pilot Length Levels", leave=False, position=1):
        pilot_length = pilot_length_array[pilot_length_index]

        theta = all_theta[sample_index, pilot_length_index, :pilot_length]
        received_signal = all_received_signal[sample_index, pilot_length_index, snr_db_index, :pilot_length]

        # Compute noise
        snr = 10 ** (snr_db / 10)
        signal_power = np.mean(np.abs(np.matmul(theta, z))**2)
        noise_power = signal_power / snr

        # Run CASBL
        gamma_result, z_result, gamma_history, z_history, iteration_count = casbl(
            theta, received_signal, noise_power, loc, max_iter=max_iter
        )

        # Save outputs
        gamma_result_all[sample_index-sample_range[0], pilot_length_index] = gamma_result.copy()
        z_result_all[sample_index-sample_range[0], pilot_length_index] = z_result.copy()

Processing Pilot Length Levels:  50%|█████     | 3/6 [00:58<00:58, 19.36s/it]

Converged after 481 iterations



Processing Pilot Length Levels:  83%|████████▎ | 5/6 [01:36<00:18, 18.84s/it]

Converged after 387 iterations



Processing Samples:   0%|          | 1/200 [01:55<6:24:02, 115.79s/it]

...

Converged after 377 iterations



Processing Pilot Length Levels:  83%|████████▎ | 5/6 [01:35<00:19, 19.59s/it]

Converged after 466 iterations



Processing Samples: 100%|██████████| 200/200 [6:27:54<00:00, 116.37s/it]

Converged after 323 iterations


In [7]:
np.savez_compressed(
    f'{base_path}/results_generation/data/casbl_snr_db_12_pilot_length_all_set_2.npz',
    gamma_result_all=gamma_result_all,
    z_result_all=z_result_all,
)

print("All results have been saved")

All results have been saved
